In [1]:
import json
import seaborn as sns
import networkx as nx
import os
from networkx.algorithms import community
from Var_Redes import *
import freeman as fm
import json
import pandas as pd
import statsmodels.api as sm

In [2]:
def make_Eignvector(rede):
    ec = nx.eigenvector_centrality(rede, max_iter=100)
    return ec

def make_Degree(rede):
    dc = nx.degree_centrality(rede)
    return dc

def make_Betweenness(rede):
    bc = nx.betweenness_centrality(rede)
    return bc

def make_Closeness(rede):
    cc = nx.closeness_centrality(rede)
    return cc

def make_AssortividadeGrau(rede):
    c = nx.degree_assortativity_coefficient(rede)
    return c

def proportion(rede, nós, vizinhos):
    return 1 / rede.degree(nós)

def local_effsize(rede, nós, vizinhos):
    s = 1
    for k in rede.neighbors(nós):
        if rede.has_edge(k, vizinhos):
            s -= proportion(rede, nós, k)
    return s

def effsize(rede, nós):
    s = 0
    for vizinhos in rede.neighbors(nós):
        s += local_effsize(rede, nós, vizinhos)
    return s

def local_constraint(rede, nós, vizinhos):
    s = proportion(rede, nós, vizinhos)
    for k in rede.neighbors(nós):
        if rede.has_edge(k, vizinhos):
            s += proportion(rede, nós, k) * proportion(rede, k, vizinhos)
    return s**2

def constraint(rede, nós):
    if rede.degree(nós) == 0:
        return 2
    s = 0
    for vizinhos in rede.neighbors(nós):
        s += local_constraint(rede, nós, vizinhos)
    return s

def hierarchy(rede, nós):
    c = constraint(rede, nós)
    nós = rede.number_of_nodes()
    s = 0
    for vizinhos in rede.neighbors(nós):
        f = local_constraint(rede, nós, vizinhos) / (c / nós)
        s += f * log(f)
    return s / (nós * log(nós))

In [3]:
class Gerador:

    def __init__(self):
        self.json_data = {}
        self.creators_game_info = {}
        self.creators_infos = {}
        self.creators_game_count = {}

    def get_data(self):
        with open('creators_filtered_copy.json') as json_file:
            self.json_data = json.load(json_file)

    def map_relations(self):

        # CREATING A JSON_DATA COPY IN ORDER TO AVOID FUTURE REDUNDANT CONNECTIONS
        json_data_copy = self.json_data.copy()

        # ITERATE TRHOUGH DEVS TO GET GAMES DONE
        for creator_id in self.json_data:

            # POPULATING DEV'S GAMES
            creator_games_ids = []
            for game in self.json_data[creator_id]['games']:
                creator_games_ids.append(game["id"])
            self.creators_game_info[creator_id] = creator_games_ids

            # DELETING DEV FROM COPY TO AVOID REDUNDANCY
            del json_data_copy[creator_id]

            # GETTING CO-WORKERS
            co_workers = []
            for creator in json_data_copy:
                # AVOIDING UNNECESSARY ITERATIONS TO SAVE RESOURCES
                if creator in co_workers:
                    break

                # MUST VERIFY ALL GAMES
                for game in json_data_copy[creator]["games"]:
                    if game["id"] in self.creators_game_info[creator_id]:
                        # JUST LIKE LINE 35
                        if creator in co_workers:
                            break
                        else:
                            co_workers.append(creator)

                                           # [rating, lista com os co_workers]
            self.creators_infos[creator_id] = [self.json_data[creator_id]["rating"] ,co_workers]
            self.creators_game_count[creator_id] = [self.json_data[creator_id]["games_count"] ,co_workers]

#           print(self.creators_relationships)

    def generate_network(self):
#         if os.path.isfile("network_data.gml"):
#             os.remove("network_data.gml")

        # CREATING GRAPH LEVEL
        f = open("network_data.gml", "w")
        f.write("graph [\n")
        f.write("   directed 0\n")
        f.write("\n")

        # CREATING NODES
        for node in self.creators_infos:
            f.write("   node [\n")
            f.write("        id " + node + "\n")
            f.write(f'       label node_{node} \n')
            f.write("   ]\n")

        # CREATE CONNECTION
        for source in self.creators_infos:
            for target in self.creators_infos[source][1]:
                f.write("   edge [\n")
                f.write("       source " + source + "\n")
                f.write("       target " + target + "\n")
                f.write("   ]\n")

        f.write("]")
        f.close()


In [4]:
gerador = Gerador()
gerador.get_data()
gerador.map_relations()
gerador.generate_network()

In [5]:
g = fm.load('network_data.gml')
# Deixar os nós menores e fazer os nomes deles
# aparecer apenas quando o mouse passa por cima.
# g.set_all_nodes(size=10, labpos='hover')

# Deixar as arestas semi-transparentes para
# a visualização ficar um pouco mais leve.
# g.set_all_edges(color=(0, 0, 0, 0.25))

# Usar um algoritmo especial para posicionar
# os nós de acordo com grupos.
# g.move('kamada_kawai')

# g.draw()

In [6]:
# com = list(community.girvan_newman(g))

In [7]:
creators_eigens = make_Eignvector(g)

In [8]:
creators_art = list(nx.articulation_points(g))

In [9]:
print(creators_art)

[11612, 134, 1119, 1954, 12312, 1057, 11910, 2293, 1902, 11345, 1184, 12349, 6420, 9314, 8891, 10744, 12610, 1332, 11949, 15153, 15967, 10221, 1555, 4141, 1158, 1157, 228, 994, 10197, 10198, 16456, 1286, 1425, 3114, 14013, 10913, 9153, 20676, 11366, 13260, 16741, 10455, 8861, 13977, 16553, 20512, 11205, 15979, 14184, 9658, 15004, 12094, 6029, 5847, 10342, 1296, 4616, 976, 60, 12613, 21939, 13418, 1682, 10861, 10998, 7787, 6051, 5338, 5449, 5450, 10205, 563, 4104, 790, 20444, 2291, 5751, 11962, 11966, 6514, 1035, 609, 612, 11570, 13916, 13917, 3044, 4556, 5162, 6329, 6522, 14283, 12376, 12377, 6043, 11709, 18439, 1352, 7898, 11460, 14162, 11382, 12381, 1818, 4327, 12676, 10060, 1646, 11743, 634, 7617, 9935, 6561, 15495, 601, 11318, 4590, 4349, 5743, 21947, 10579, 21861, 10063, 3315, 17312, 16690, 3136, 9908, 18492, 10490, 10491, 2266, 1643, 11334, 4341, 3194, 2751, 289, 929, 3080, 1006, 1003, 5194, 1002, 266, 371, 72, 11514, 20630, 10554, 631, 4204, 10619, 14092, 11680, 8405, 8406, 2479

In [10]:
creators_artpoints = {}

for source in gerador.creators_infos:
    if int(source) in creators_art:
        creators_artpoints[source] = 1
    else:
        creators_artpoints[source] = 0

print(creators_artpoints)

{'1': 0, '10': 0, '100': 0, '1000': 0, '10001': 0, '10002': 0, '10003': 0, '10004': 0, '10005': 0, '10008': 0, '10009': 0, '1001': 0, '10011': 0, '10012': 0, '10013': 0, '10014': 1, '10015': 0, '10016': 0, '10017': 0, '10018': 0, '10019': 0, '1002': 1, '10020': 0, '10022': 0, '10023': 0, '10024': 1, '10025': 0, '10026': 0, '10027': 0, '10028': 0, '10029': 0, '1003': 1, '10030': 0, '10031': 0, '10032': 0, '10033': 0, '10034': 0, '10036': 0, '10037': 0, '10038': 0, '10039': 0, '1004': 0, '10041': 0, '10042': 1, '10044': 0, '10046': 0, '10047': 0, '10048': 0, '10049': 0, '1005': 0, '10050': 0, '10051': 0, '10052': 0, '10053': 0, '10054': 0, '10055': 0, '10058': 0, '10059': 0, '1006': 1, '10060': 1, '10062': 0, '10063': 1, '10064': 0, '10065': 0, '10066': 0, '10068': 0, '10069': 0, '1007': 0, '10071': 0, '10072': 0, '10073': 1, '10074': 0, '10075': 0, '10076': 0, '10077': 0, '10078': 0, '1008': 0, '10080': 0, '10081': 0, '10082': 0, '10083': 0, '10084': 0, '10085': 0, '10087': 0, '10088': 

In [11]:
creators_ratings = {}

for source in gerador.creators_infos:
    creators_ratings[source] = gerador.creators_infos[source][0]
    
print(creators_ratings)

{'1': 4.36, '10': 4.386666666666667, '100': 4.15, '1000': 4.3500000000000005, '10001': 2.3000000000000003, '10002': 3.0525, '10003': 3.0525, '10004': 3.56, '10005': 3.56, '10008': 0.0, '10009': 0.0, '1001': 4.234999999999999, '10011': 0.0, '10012': 0.0, '10013': 4.22, '10014': 2.7399999999999998, '10015': 4.07, '10016': 4.07, '10017': 3.6025, '10018': 3.79, '10019': 3.0233333333333334, '1002': 3.345, '10020': 3.6966666666666668, '10022': 1.205, '10023': 0.0, '10024': 0.0, '10025': 1.6, '10026': 1.6, '10027': 1.0666666666666667, '10028': 1.6, '10029': 1.6, '1003': 3.448333333333333, '10030': 1.6, '10031': 0.8, '10032': 1.6, '10033': 1.6, '10034': 1.6, '10036': 0.0, '10037': 0.0, '10038': 1.05, '10039': 1.7875, '1004': 4.07, '10041': 0.0, '10042': 0.0, '10044': 0.0, '10046': 0.0, '10047': 0.0, '10048': 0.0, '10049': 0.0, '1005': 4.07, '10050': 0.0, '10051': 0.858, '10052': 0.0, '10053': 4.04, '10054': 3.67, '10055': 0.0, '10058': 0.6916666666666668, '10059': 1.3833333333333335, '1006': 2

In [12]:
creators_game_count = {}

for source in gerador.creators_game_count:
    creators_game_count[source] = gerador.creators_game_count[source][0]
    
# print(creators_game_count)

In [13]:
# print(creators_eigens)

In [14]:
eigens = []
for key, value in creators_eigens.items():
    eigens.append(value)

In [15]:
ratings = []
for key, value in creators_ratings.items():
    ratings.append(value)

In [16]:
game_count = []
for key, value in creators_game_count.items():
    game_count.append(value)

In [17]:
art_points =[]
for key, value in creators_artpoints.items():
    art_points.append(value)

In [18]:
# Primeiro parâmetro: Y
# Segundo parâmetro: X1, ..., Xn (não esqueça os colchetes duplos)
model = sm.OLS(ratings, eigens)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.002
Model:                            OLS   Adj. R-squared (uncentered):              0.002
Method:                 Least Squares   F-statistic:                              45.50
Date:                Mon, 16 Nov 2020   Prob (F-statistic):                    1.57e-11
Time:                        10:45:41   Log-Likelihood:                         -46030.
No. Observations:               19556   AIC:                                  9.206e+04
Df Residuals:                   19555   BIC:                                  9.207e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            17.1794      2.547      6.745      0.000      12.187      22.171
==============================================================================
Omnibus:                    84998.238   Durbin-Watson:                   0.452
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2358.532
Skew:                           0.075   Prob(JB):                         0.00
Kurtosis:                       1.305   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [19]:
# Primeiro parâmetro: Y
# Segundo parâmetro: X1, ..., Xn (não esqueça os colchetes duplos)
model = sm.OLS(ratings, game_count)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.436
Model:                            OLS   Adj. R-squared (uncentered):              0.436
Method:                 Least Squares   F-statistic:                          1.512e+04
Date:                Mon, 16 Nov 2020   Prob (F-statistic):                        0.00
Time:                        10:45:41   Log-Likelihood:                         -40451.
No. Observations:               19556   AIC:                                  8.090e+04
Df Residuals:                   19555   BIC:                                  8.091e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.7017      0.006    122.975      0.000       0.690       0.713
==============================================================================
Omnibus:                     9074.513   Durbin-Watson:                   1.080
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1239.239
Skew:                           0.264   Prob(JB):                    7.99e-270
Kurtosis:                       1.885   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [22]:
# Primeiro parâmetro: Y
# Segundo parâmetro: X1, ..., Xn (não esqueça os colchetes duplos)
model = sm.OLS(ratings, art_points)
result = model.fit()
result.summary()
